In [1]:
#NOTE:
"uriyas model total emotions: happy, sad, disgust, fear, neutral, anger, surprise"
"use for valence: happy/sad"
"if needed, he'll use for arousal: neutral/suprise"
#THEN, for me:
#map the emotions to (x,y) coordinates of valence and arousal, then multiply it by the probability of the emotion, then take the weighted average and get the reslting x,y coordinates
#valence and arousal are the two dimensions of the circumplex model and the resulting scores should get put in one of the 9(nuetral plus 8 categories) categories

"if needed, he'll use for arousal: neutral/suprise"

# GoEmotions: Testing Input to the Model, Adding Softmax Layer, Conversion Function for getting segment Label, Getting final Label

# 1. Loading Model

In [2]:
import transformers
import tensorflow as tf
import torch

c:\Users\mlar5\anaconda3\envs\CudaTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#load the model monologg/bert-base-cased-goemotions-original from huggingface
model = transformers.AutoModelForSequenceClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")
tokenizer = transformers.AutoTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [433]:
#test the model on a sentence
sentence = "I am so happy"
tokens = tokenizer.encode(sentence, return_tensors="pt")
tokens = tokens.cuda()
print(tokens)
result = model(tokens)

tensor([[ 101,  146, 1821, 1177, 2816,  102]], device='cuda:0')


# 2. Playing around/testing model (skip)
this is to figure out what will be needed making softmax, and determining the classes

In [5]:
#determine the max size of the input the model can handle
max_length = model.config.max_position_embeddings
print(max_length)

512


In [6]:
#identify the top 3 emotion with the highest probability
emotion = result.logits
emotion = emotion.cpu().detach().numpy()
emotion = emotion[0]
emotion = emotion.argsort()[-3:][::-1]
emotion = emotion.tolist()
print(emotion)

[17, 0, 13]


In [7]:
#convert model.config.id2label to a list of emotions, where the key is the index of the emotion
emotion_dict = model.config.id2label

In [8]:
#identify the label of top 3 emotions from emotion list
for i in emotion:
    print(emotion_dict[i])


joy
admiration
excitement


In [9]:
#identify the emotion with the highest probability
emotion = result.logits.argmax()
print(emotion)
#which emotion is that?
print(model.config.id2label[emotion.item()])

tensor(17, device='cuda:0')
joy


In [10]:
emotion_dict.values()
emotionGroups=[]

In [11]:
emotionSet = set(emotion_dict.values())

In [12]:
approvedMoods=['neutral']

# 3. Softmax

### Function

In [458]:
def getSoftmax(model,tokenizer, tokens = None, sentence=None, n=3,printRawScores=False, printTopN=False):
    if tokens is None:
        tokens = tokenizer.encode(sentence, return_tensors="pt")
        tokens = tokens.cuda()
    result = model(tokens)
    emotion = result.logits
    emotion = emotion.cpu().detach().numpy()
    emotion = emotion[0]
    softmax = tf.nn.softmax(emotion)
    #convert to numpy array
    softmax = softmax.numpy()
    if printRawScores:
        print(softmax)
    
    if printTopN:
        emotion = emotion.argsort()[-n:][::-1]
        emotion = emotion.tolist()
        printTopEmotions(emotion,model, softmax)
    return softmax


def printTopEmotions(emotion, model, softmax):
    
    #identify the label of top n emotions from emotion list
    #softmax is in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    id=0
    emotion_dict = model.config.id2label
    for i in emotion:
        print(emotion_dict[i])
        print(softmax[emotion[id]]*100,"%")
        id+=1
    return

### sentences from songs to test with

In [14]:
unclear2 = "'Cause you see people, people, people, people Don't really know you (They don't really know you) They don't really know you 'Cause you see people, people, people They don't really know you (Mmm) They don't really know—"
unclear = "I've been drinking more alcohol for the past five days Did you check on me? Now, did you look for me? I walked in the room, eyes are red and I don't smoke banga Did you check on me? (Did you check on me?) Now, did you notice me?"

In [15]:
braggingLyrics = "She just hit my phone, she said, 'Tecca, you a winner' (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I’m a star I just wanna ball with my guys Drive off the Wock', he just caught a DUI He think he know the answers, nigga, like he Bill Nye"

In [16]:
sadMetaphoricLyrics = "Hello darkness, my old friend, I've come to talk with you again, because a vision softly creeping, left its seeds while I was sleeping."

In [17]:
goEasyOnMeSad = "There ain't no gold in this river That I've been washin' my hands in forever I know there is hope in these waters But I can't bring myself to swim When I am drowning in this silence Baby, let me in. Go easy on me, baby I was still a child Didn't get the chance to Feel the world around me I had no time to choose what I chose to do So go easy on me"

### Results

In [368]:
softmaxtestUnclear=getSoftmax(model, sentence = unclear, printTopN=True)

curiosity
99.90363121032715 %
neutral
0.034640790545381606 %
confusion
0.019302892906125635 %


In [369]:
softmaxtestUnclear2=getSoftmax(model,sentence =unclear2, printTopN=True)

neutral
99.9980092048645 %
annoyance
0.00058388282013766 %
disappointment
0.0003420888106120401 %


In [370]:
softmaxtestBraggingLyrics=getSoftmax(model,sentence =braggingLyrics, printTopN=True)

neutral
75.65684914588928 %
joy
16.482388973236084 %
excitement
4.176769405603409 %


In [371]:
softmaxtestsadMetaphoricLyrics=getSoftmax(model,sentence =sadMetaphoricLyrics, printTopN=True)

neutral
99.99864101409912 %
annoyance
0.00020481845695030643 %
caring
0.00011883536217283108 %


In [373]:
softmaxtestgoEasyOnMeSad=getSoftmax(model,sentence =goEasyOnMeSad, printTopN=True)

disappointment
63.13481330871582 %
realization
14.627596735954285 %
sadness
14.258602261543274 %


# 3. SoftMax Recap:

1. There seems to be a dominance of neutral while the trailing classes seem to be more relevant. This does make sense considering the fact that not every lyric is emotionally charged.  

- However, this does raise the question, "How do we handle neutral?"

- similarly, "Do we want neutral as a resulting category?

2. Songs that are reliant on metaphores or to create emotion will not be captured via sentiment analysis

- In these cases, we would want the linear model to give results
- perhaps if this is specifially reated neutral we should rely on the linear classificiaton model

# 4. Class Conversion Without Special Neutral Considerations

Here we will:
1. scale emotions to (valence, energy) based on the valence-arousal mood wheel
2. multiply that value by the percent
3. add up x values as well as y values
4. place resulting values into the 8 main mood categories to get a class
5. if close to 0, label it neutral instead
    - If neutral, perhaps use as indicator to rely on 2nd model for classification

NOTE: Some words did not have a perfect point plotted, but there were some synonyms
- approval was mapped with satisfied
- caring was with passionate
- confused with a lessened amount of frustrated (1/2)
- curious was mapped with interested

### Mood setup verification (skip)

In [409]:
emotion_dict = model.config.id2label
for key in range(len(emotion_dict)):
    print(emotion_dict[key])

admiration
amusement
anger
annoyance
approval
caring
confusion
curiosity
desire
disappointment
disapproval
disgust
embarrassment
excitement
fear
gratitude
grief
joy
love
nervousness
optimism
pride
realization
relief
remorse
sadness
surprise
neutral


In [24]:
# make a dictionary mapping all emotions to valence and arousal values
# valence is the x axis, arousal is the y axis of the circumplex model
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,-.3),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}


In [25]:
for i in range(len(emotion_dict)):
    print(emotionsAsValenceArousal[emotion_dict[i]])

(0.6, 0.4)
(0.6, 0.2)
(-0.8, 0.6)
(-0.6, 0.6)
(0.8, 0.6)
(0.6, -0.2)
(-0.2, 0.2)
(0, 0.4)
(0.6, 0.6)
(-0.6, -0.6)
(-0.8, 0.65)
(-0.8, 0.2)
(-0.6, 0.4)
(0.6, 0.8)
(-0.6, 0.8)
(0.6, -0.6)
(-0.6, -0.8)
(0.8, 0.2)
(0.8, 0.4)
(-0.4, 0.6)
(0.6, -0.3)
(0.6, 0.1)
(0.2, 0.2)
(0.4, -0.4)
(-0.6, -0.4)
(-0.8, -0.2)
(0.2, 0.6)
(0, 0)


### Function

In [26]:
emotionsAsValenceArousal= { 'admiration':(.6,.4),'amusement':(.6,.2),'anger':(-.8,.6),'annoyance':(-.6,.6),'approval':(.8,.6),'caring':(.6,-.2),'confusion':(-.2,.2),'curiosity':(0,.4),'desire':(.6,.6),'despair':(-.8,-.6),'disappointment':(-.6,-.6),'disapproval':(-.8,.65),'disgust':(-.8,.2),'embarrassment':(-.6,.4),'envy':(-.6,.4),'excitement':(.6,.8),'fear':(-.6,.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'gratitude':(.6,-.6),'grief':(-.6,-.8),'joy':(.8,.2),'love':(.8,.4),'nervousness':(-.4,.6),'optimism':(.6,.2),'pride':(.6,.1),'realization':(.2,.2),'relief':(.4,-.4),'remorse':(-.6,-.4),'sadness':(-.8,-.2),'surprise':(.2,.6),'neutral':(0,0)}
#moods=['energetic','excited','happy','relaxed','calm','depressed','sad','anxious']

In [481]:

def determineMoodLabel(valence,arousal):
    #determine the diagonal of the circumplex model that the valence and arousal scores fall on
    #MAKE 2 BOXES OF THE CIRCUMPLEX MODEL A MOOD 

    energetic =   -0.5<valence<0.5 and arousal>0.5
    happy =       valence>0.5 and -.5<arousal<0.5
    calm =       -0.5<valence<0.5 and arousal<-0.5
    sad =         valence<-0.5 and -.5<arousal<0.5

    excited =   not (happy or energetic) and valence>0 and arousal>0
    relaxed =   not (calm or happy) and valence>0 and arousal<0
    depressed = not (calm or sad) and valence<0 and arousal<0
    anxious =   not (energetic or sad) and valence<0 and arousal>0


    if energetic:
        mood='energetic'
    elif happy:
        mood='happy'
    elif calm:
        mood='calm'
    elif sad:
        mood='sad'
    elif excited:
        mood='excited'
    elif relaxed:
        mood='relaxed'
    elif depressed:
        mood='depressed'
    elif anxious:
        mood='anxious'
    else:
        mood='neutral'
    return mood     

def convertScoresToLabels(softmaxScoresPerHeader,headerFreqs, emotionsAsValenceArousal,emotion_dict,disregardNeutral = True, printValenceArousal=False,printTopChunkEmotions=False):
    #convert the softmax scores to a valence and arousal score
    #softmax scores are in the order of the values in emotion_dict so we can use emotion[id] to get the softmax value
    valence=0
    arousal=0
    #find the key in emotion_dict that corresponds to neutral
    neuturalKey = [key for key, value in emotion_dict.items() if value == 'neutral'][0]
    for key, softmaxScores in softmaxScoresPerHeader.items():
        #check if neutral is the highest softmax score
        if disregardNeutral and neuturalKey==softmaxScores.argmax():
            continue
        else:
            #multiply the softmax score by the valence and arousal values and add to the total valence and arousal
            #do this for the number in the headerFreqs dictionary
            for i in range(headerFreqs[key]):
                id=0
                for i in softmaxScores:
                    valence+=i*emotionsAsValenceArousal[emotion_dict[id]][0]
                    arousal+=i*emotionsAsValenceArousal[emotion_dict[id]][1]
                    id+=1
    mood =determineMoodLabel(valence,arousal)
    if printValenceArousal:
        print("Valence: ",valence)
        print("Arousal: ",arousal)
    return mood

### Broken function but cleaner code (skip)

In [381]:
"""
def determineMoodLabel(valence, arousal):
    conditions = {
        'energetic': (-0.5 < valence < 0.5) and (arousal > 0.5),
        'happy': (valence > 0.5) and (-0.5 < arousal < 0.5),
        'calm': (-0.5 < valence < 0.5) and (arousal < -0.5),
        'sad': (valence < -0.5) and (-0.5 < arousal < 0.5),
    }
    conditions['excited']= (valence > 0) and (arousal > 0) and not ((conditions.get('happy') == True) or (conditions.get('energetic') == True))
    conditions['relaxed'] =(valence > 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('happy') == True)),
    conditions['depressed']= (valence < 0) and (arousal < 0) and not ((conditions.get('calm')==True) or (conditions.get('sad') == True)),
    conditions['anxious']= (valence < 0) and (arousal > 0) and not ((conditions.get('energetic') == True) or (conditions.get('sad') == True))

    for mood, condition in conditions.items():
        #print(mood)
        if condition:
            if condition:
            #print(condition)
                return mood
    return 'neutral'
"""
None

### Results

In [29]:
determineMoodLabel(1,1)

'excited'

In [30]:
convertScoresToLabels(softmaxtestUnclear, emotionsAsValenceArousal,emotion_dict,printValenceArousal=True)

Valence:  -0.00014550616
Arousal:  0.39976904


'anxious'

In [31]:
convertScoresToLabels(softmaxtestUnclear2, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -6.28967e-06
Arousal:  4.1340345e-06


'anxious'

In [32]:
convertScoresToLabels(softmaxtestBraggingLyrics, emotionsAsValenceArousal,emotion_dict,True)

Valence:  0.17935042
Arousal:  0.0792139


'excited'

In [33]:
convertScoresToLabels(softmaxtestsadMetaphoricLyrics, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -1.578913e-06
Arousal:  2.6779267e-06


'anxious'

In [34]:
convertScoresToLabels(softmaxtestgoEasyOnMeSad, emotionsAsValenceArousal,emotion_dict,True)

Valence:  -0.4687149
Arousal:  -0.37509394


'depressed'

# 4. Class Conversion Recap:

- The code works but it tends to classify heavily towards the center of the coordinates
- NOTE: This could be because it was just a few small samples, which may not reflect the overall distribution of the data
- As an alternative, we could remove the neutral score and apply softmax to the remaining classes
- OR, I could just exclude the inputs that result in overwhemingly neutral, and then continue to the next chunk of lyrics until I find all chunks that are not neutral and then get the average of those to get the final classification

# 5. Class Conversion With Special Neutral Considerations

First, in 6 and 7, we will try to get the average of the classes that are not overwhelmingly neutral to get final label.

If that doesnt work, here we will drop the neutral score and apply softmax to the remaining classes

- This would allow us to multiply the coordinates without shrinking the values due to the neutral score
- Perhaps this would allow us to reach the classes that are further away from the center
- If not, we could shift the boundaries of the classes closer to the center

# 6. Breaking up Lyrics to fit into model

In order to fit the lyrics into the model, we need to break up the lyrics. There are various ways to do this. We will try a few different methods and see which one works best.


- (Best) We could break it up by headers. Below are the various headers that we could use to break up the lyrics
https://genius.com/Genius-song-sections-and-headers-guide-annotated
    - in general, the intro is least emotional, the chorus is most emotional, and the verses are in between
    - This could mean we could skip the intro/outro and rely on the chorus and verses (maybe just the chorus when need fast results)

Some other ideas:
- We could break it up by sentences (some songs have no punctuation, so this may not work)
- We could break it up by line

### Song Samples used in testing (skip if not interested, but run cells)

Song sample 1.

This one is intentionally vague, but it is more relaxed or happy

In [340]:
fullLyricsLilTecca = """
[Intro]
Nah, nah, nah
Yeah, oh-oh
Yeah, oh no
We love you Tecca
Yeah, yeah
She just hit my phone, she said, "Tecca, you a winner" (That's Tec)
I just took her home, then I turn her to a sinner, yeah
Rambow

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they say that I’m a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he know the answers, nigga, like he Bill Nye
[Verse]
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they sayin’ I'm a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he got the answers, nigga, like he Bill Nye
"""

In [339]:
fullLyricsLilTeccaTripleVerse = """
[Intro]
Nah, nah, nah
Yeah, oh-oh
Yeah, oh no
We love you Tecca
Yeah, yeah
She just hit my phone, she said, "Tecca, you a winner" (That's Tec)
I just took her home, then I turn her to a sinner, yeah
Rambow

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they say that I’m a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he know the answers, nigga, like he Bill Nye
[Verse]
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor
Bitch, I feel like A.I., nigga, never need to practice
I got two clips 'cause that chopper automatic
And I started talkin’ shit, I got tired of this rappin'
But I been making' hits, most these niggas, they been nappin'
And my lyrics hit her hard, like her nigga, he was stabbin'
LSD hit his brain, yeah, that nigga, he been tabbin'
I just got your shorty, yeah, my niggas, we been tappin'
Walk to the bank, Chief Keef, I be laughin'
Red, blue, yellow, nigga, you could pick your Power Ranger
I could fuck, but I won’t date ya
Getting bands, but I can’t save her
Fast nigga, but I can't chase her
360, quick scope, FaZe him
God-mode, aim-bot, laser
Cut you off, I got that razor

[Chorus]
She just hit my phone, she said, "Tecca, you a winner" (Yeah)
Took her home, then I turn her to a sinner, yeah
I just got a check, I ate lobster for dinner
She like my style, she tryna turn me to her nigga, yeah
I just woke up, now they sayin’ I'm a star
I just wanna ball with my guys
Drive off the Wock', he just caught a DUI
He think he got the answers, nigga, like he Bill Nye
"""

Song sample 2.

This one is more sad, and it is Go easy on me by Adele



In [36]:
fullLyricsAdele =""

### 6.0.1. Determining split when tokenizing lyrics exceeds 512 (model input limit) (skip)

The goal should be to split it as evenly as possible.

If over limit but under 1024, we can split it into 2 chunks, either at the nerest line break or paragraph break to the middle of the chunk

If it is bigger, we can divide by 512 to get n and then split it as smooth as possible into n+1 chunks

In [322]:
'''
def breakUpLargeLyricChunks1(lyricsChunkString, returnLyricsSegments=False):
    songSegments=[lyricsChunkString]
    tokenSegments = []

    #convert the string to a list of lines in the string and also store the start and end index of each line
    lines = lyricsChunkString.splitlines()

    tokens = tokenizer.encode(lyricsChunkString)
    max_length = 512# model.config.max_position_embeddings is always 512 and it does not change

    if len(tokens) < max_length and torch.cuda.is_available():
        tokenSegment = tokenizer.encode(segment, return_tensors="pt").cuda()
        tokenSegments.append(tokenSegment)
    elif len(tokens) < max_length:
        tokenSegment= tokenizer.encode(segment, return_tensors="pt")
        tokenSegments.append(tokenSegment)
    else:
        songSegments = []
        songSegmentCount = int(len(tokens)/max_length)+1+1
        averageSegmentSize = int(len(lines)/songSegmentCount)

        #for the number of songSegmentCount, decode the tokens by the average segment size to get the string
        for i in range(songSegmentCount):
            #decode the tokens by the average segment size to get the string
            startidx=averageSegmentSize*(i)
            endidx=averageSegmentSize*(i+1)

            #if it is the last i then set the endidx to the last line
            if i == songSegmentCount-1:
                endidx = len(lines)

            #make a segment of the lyrics combining the range of lines
            segment = " ".join(lines[startidx:endidx])

            songSegments.append(segment)

            #if cuda is available,  add to tokenSegments
            if torch.cuda.is_available():
                tokenSegment = tokenizer.encode(segment, return_tensors="pt").cuda()
                if len(tokenSegment) < max_length:
                    tokenSegments.append(tokenSegment)
                else:
                    print("segment is too long")

            else:
                tokenSegment= tokenizer.encode(segment, return_tensors="pt")
                if len(tokenSegment) < max_length:
                    tokenSegments.append(tokenSegment)
                else:
                    print("segment is too long")

    #NOTE: I should add a check for the chunk size to make sure it is still not over the max length
    if returnLyricsSegments:
        return tokenSegments, songSegments
    else:
        return tokenSegments
        '''
None

### 6.0.2 Making function 6.0.1 more efficient

In [445]:
def breakUpLargeLyricChunks(lyricsChunkString, lines,tokenizer, max_length=512, device="cuda", returnLyricsSegments=False):
    #lines = lyricsChunkString.splitlines()  # split the lyrics into lines
    segments = []  # store the lyrics segments
    token_segments = []  # store the tokenized segments as tensors
    #print(type(lyricsChunkString))
    token_segment = tokenizer.encode(lyricsChunkString, return_tensors="pt").to(device)

    if len(token_segment[0]) <= max_length:
        token_segment = token_segment.unsqueeze(0)
        token_segments.append(token_segment)
        segments.append(lyricsChunkString)
    else:
        # calculate the average number of lines per segment. Add +2 to ensure segments are not still too long
        avg_lines_per_segment = len(lines) // ((len(token_segment[0]) // max_length) + 2)

        # loop through the lines and group them into segments of roughly the same length
        for start_idx in range(0, len(lines), avg_lines_per_segment):
            end_idx = start_idx + avg_lines_per_segment

            smallLastChunk = end_idx >= len(lines)-2
            
            if smallLastChunk:
                segment = " ".join(lines[start_idx:])
            else:
                segment = " ".join(lines[start_idx:end_idx])
            segments.append(segment)

            # tokenize the segment and convert to tensor
            token_segment = tokenizer.encode(segment, return_tensors="pt").to(device)
            token_segment = token_segment.unsqueeze(0)
            token_segments.append(token_segment)
            #NOTE: ^^ If I use batch_encode_plus, I can get the tokenized segments as a list of tensors in one step
            #I would just have to do it after the loop. 
            #Since it is a small list though, I don't think it will make a difference in this case

            if smallLastChunk:
                #this is the last segment early, so break out of the loop
                break

    if returnLyricsSegments:  
        return token_segments, segments
    else:
        return token_segments


In [392]:
x, xlyrics= breakUpLargeLyricChunks(fullLyricsLilTecca, fullLyricsLilTecca.splitlines(),  tokenizer,returnLyricsSegments=True )#, max_length=512, device="cuda", returnLyricsSegments=False)

In [393]:
len(x)

3

In [394]:
xlyrics[2]

'I could fuck, but I won’t date ya Getting bands, but I can’t save her Fast nigga, but I can\'t chase her 360, quick scope, FaZe him God-mode, aim-bot, laser Cut you off, I got that razor  [Chorus] She just hit my phone, she said, "Tecca, you a winner" (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they sayin’ I\'m a star I just wanna ball with my guys Drive off the Wock\', he just caught a DUI He think he got the answers, nigga, like he Bill Nye'

## 6.1 Breaking up by headers

Now that we accounted for the limit, we can break up the lyrics by headers and split large chunks if needed

### old function (skip)

In [364]:
'''
# input: a string of whole song
# output: a dictionary of with header values and a list of tensors (sometmes more than 1 item) for each header chunk
def breakUpSongByHeaders(fullSongLyricsString, tokenizer, max_length=512, device="cuda",  returnSongSegments=False):
    songSegmentsDict = {}
    tokenSegmentsDict = {}
    headerFreqsDict = {}

    #split the song into a list of lines
    lines = fullSongLyricsString.splitlines()[1:]
    #find the lines that start with [ and end with ]
    headerLinesIndex = [i for i, s in enumerate(lines) if "[" in s and "]" in s]
    #update the lines function by removing the [ and ] from the lines
    lines = [line.replace("[","").replace("]","") for line in lines]

    for index in headerLinesIndex:
        print(lines[index])
    #create a dictionary of the header lines and the lines that follow before the next header line
    for i in range(len(headerLinesIndex)):
        if lines[headerLinesIndex[i]] in songSegmentsDict:
            songSegmentsDict[lines[headerLinesIndex[i]]][0] += 1
        elif i == len(headerLinesIndex)-1:
            songSegmentsDict[lines[headerLinesIndex[i]]] = [1," ".join(lines[headerLinesIndex[i]+1:]),lines[headerLinesIndex[i]+1:]]
        else:
            songSegmentsDict[lines[headerLinesIndex[i]]] = [1," ".join(lines[headerLinesIndex[i]+1:headerLinesIndex[i+1]]),lines[headerLinesIndex[i]+1:headerLinesIndex[i+1]]]
    for header, lyrics in songSegmentsDict.items():
        if returnSongSegments:
            tokenSegmentsDict[header],subLyrics = breakUpLargeLyricChunks(lyrics[1],lyrics[2],tokenizer,returnLyricsSegments=returnSongSegments,max_length=max_length, device=device)
            songSegmentsDict[header]=subLyrics
        else:
            tokenSegmentsDict[header] = breakUpLargeLyricChunks(lyrics[1],lyrics[2],tokenizer,returnLyricsSegments=returnSongSegments,max_length=max_length, device=device)
        headerFreqsDict[header] = lyrics[0]
        
    if returnSongSegments:
        return tokenSegmentsDict,headerFreqsDict,songSegmentsDict
    else:
        return tokenSegmentsDict,headerFreqsDict
'''
None

### Slightly cleaner code

In [365]:
# input: a string of whole song
# output: a dictionary of with header values and a list of tensors (sometmes more than 1 item) for each header chunk
def breakUpSongByHeaders(fullSongLyricsString, tokenizer, max_length=512, device="cuda",  returnSongSegments=False):
    songSegmentsDict = {}
    tokenSegmentsDict = {}
    headerFreqsDict = {}

    #split the song into a list of lines
    lines = fullSongLyricsString.splitlines()
    #find the lines that start with [ and end with ]
    headerLinesIndex = [i for i, line in enumerate(lines) if line.startswith('[') and line.endswith(']')]

    for i in range(len(headerLinesIndex)):
        header_line = lines[headerLinesIndex[i]][1:-1]  # remove square brackets
        if header_line in songSegmentsDict:
            songSegmentsDict[header_line][0] += 1
        elif i == len(headerLinesIndex)-1:
            songSegmentsDict[header_line] = [1, " ".join(lines[headerLinesIndex[i]+1:]), lines[headerLinesIndex[i]+1:]]
        else:
            songSegmentsDict[header_line] = [1, " ".join(lines[headerLinesIndex[i]+1:headerLinesIndex[i+1]]), lines[headerLinesIndex[i]+1:headerLinesIndex[i+1]]]

    for header, lyrics in songSegmentsDict.items():
        if returnSongSegments:
            tokenSegmentsDict[header],subLyrics = breakUpLargeLyricChunks(lyrics[1],lyrics[2],tokenizer,returnLyricsSegments=returnSongSegments,max_length=max_length, device=device)
            songSegmentsDict[header]=subLyrics
        else:
            tokenSegmentsDict[header] = breakUpLargeLyricChunks(lyrics[1],lyrics[2],tokenizer,returnLyricsSegments=returnSongSegments,max_length=max_length, device=device)
        headerFreqsDict[header] = lyrics[0]

    if returnSongSegments:
        return tokenSegmentsDict,headerFreqsDict,songSegmentsDict
    else:
        return tokenSegmentsDict,headerFreqsDict


### Testing Function

In [395]:
songChunks,freqs =breakUpSongByHeaders(fullLyricsLilTecca,tokenizer)

In [396]:
len(songChunks['Verse'])

1

In [397]:
songChunks,freqs,songSegs =breakUpSongByHeaders(fullLyricsLilTeccaTripleVerse,tokenizer,returnSongSegments=True)

In [398]:
len(songSegs['Verse'])

3

In [399]:
songSegs['Intro']

['Nah, nah, nah Yeah, oh-oh Yeah, oh no We love you Tecca Yeah, yeah She just hit my phone, she said, "Tecca, you a winner" (That\'s Tec) I just took her home, then I turn her to a sinner, yeah Rambow ']

In [400]:
songSegs

{'Intro': ['Nah, nah, nah Yeah, oh-oh Yeah, oh no We love you Tecca Yeah, yeah She just hit my phone, she said, "Tecca, you a winner" (That\'s Tec) I just took her home, then I turn her to a sinner, yeah Rambow '],
 'Chorus': ['She just hit my phone, she said, "Tecca, you a winner" (Yeah) Took her home, then I turn her to a sinner, yeah I just got a check, I ate lobster for dinner She like my style, she tryna turn me to her nigga, yeah I just woke up, now they say that I’m a star I just wanna ball with my guys Drive off the Wock\', he just caught a DUI He think he know the answers, nigga, like he Bill Nye'],
 'Verse': ["Bitch, I feel like A.I., nigga, never need to practice I got two clips 'cause that chopper automatic And I started talkin’ shit, I got tired of this rappin' But I been making' hits, most these niggas, they been nappin' And my lyrics hit her hard, like her nigga, he was stabbin' LSD hit his brain, yeah, that nigga, he been tabbin' I just got your shorty, yeah, my niggas,

In [401]:
songChunks['Intro']

[tensor([[  101, 24883,   117,  9468,  1324,   117,  9468,  1324,  2814,   117,
           9294,   118,  9294,  2814,   117,  9294,  1185,  1284,  1567,  1128,
          12008, 19495,  2814,   117,  8147,  1153,  1198,  1855,  1139,  2179,
            117,  1131,  1163,   117,   107, 12008, 19495,   117,  1128,   170,
           2981,   107,   113,  1337,   112,   188, 12008,  1665,   114,   146,
           1198,  1261,  1123,  1313,   117,  1173,   146,  1885,  1123,  1106,
            170, 11850,  2511,   117,  8147, 11447, 14251,   102]],
        device='cuda:0')]

In [402]:
freqs

{'Intro': 1, 'Chorus': 2, 'Verse': 1}

In [ ]:
count = 0
for header,tokenChunks in songChunks.items():
    for tokenChunk in tokenChunks:
    #softmax takes in a sentence but its already encoded, so make sure to add param to skip that line if its already encoded
        softmaxScores=getSoftmax(model,tokenizer,tokens=tokenChunk, printTopN=True)
        count += 1

In [405]:
print(count)

5


In [417]:
softmaxScores+softmaxScores

<tf.Tensor: shape=(28,), dtype=float32, numpy=
array([4.71507868e-04, 6.64192426e-04, 1.88800669e+00, 3.85590866e-02,
       3.15132347e-05, 1.03654747e-03, 4.55209432e-04, 1.04930070e-04,
       1.36494738e-04, 1.86057400e-03, 2.81385286e-03, 1.09431548e-02,
       2.42148017e-04, 5.44508148e-05, 3.08011728e-03, 7.79700422e-05,
       5.52225101e-04, 3.27373593e-04, 3.33590171e-04, 6.30960785e-05,
       3.88773886e-04, 1.23360354e-04, 1.20458135e-04, 4.15803333e-05,
       1.43959711e-04, 4.34775429e-04, 2.44906842e-04, 4.86875474e-02],
      dtype=float32)>

In [472]:
#covert softmax scores to numpy array
softmaxScores = softmaxScores.numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'

# Recap: The function now breaks up the lyrics by headers and splits large chunks if needed. While the example of the verse with a large chunk I showed was storing the same lyrics 3x, that's just becase I copy-pasted the lyrics. In reality, the lyrics would've been different but successfully broken up into chunks if needed.

# 7. Final Function For Inferences based on Lyrics

NOTE 1 - if top rating is neutral, Either we should 
1. discard segment and move on to next segment
2. Take the second highest rating and use that as the sgement label

Note 2 - If all ratings are neutral, we should use the linear model to get the final label

Edge cases taken into account:
- the song segments that are too long are divided by n+1 chunks where n is the number of chunks it should take to fit into the model. This is because the songs are split by lines so to avoid a long line making the chunk too big, we split it into n+1 chunks
- factoring in the frequencies of headers in the song to determine the weights of the headers

Edge cases not taken into account:
- adding periods at the end of lines
- adding a check for songs with no headers
- adding a check for songs not in english
- considering random brackets that are not headers!


In [489]:
def getMoodLabelFromLyrics(lyrics,model, tokenizer, emotion_dict, emotionsAsValenceArousal,disregardNeutral=True, printRawScores=False, printTopN=False,topScoresToPrint=3,max_length=512, device="cuda",  returnSongSegments=False):
    softmaxScoresPerHeader = {}
    #part 1 - break up the lyrics into chunks and get the tokens
    if returnSongSegments:
        songTokenChunks,freqs,songSegs =breakUpSongByHeaders(lyrics,tokenizer,returnSongSegments=returnSongSegments,max_length=max_length, device=device)
    else:
        songTokenChunks,freqs =breakUpSongByHeaders(lyrics,tokenizer,returnSongSegments=returnSongSegments,max_length=max_length, device=device)

    #part 2 - get the softmax score for each chunk

    #softmax scores returns COMBINED SINGLE LABEL -- MAYBE TRY MULTIPLE LABELS AND TAKE THE MOST COMMON
    for header,tokenChunksPerHeaders in songTokenChunks.items():
        for tokenChunks in tokenChunksPerHeaders:
            for tokenChunk in tokenChunks:
                if header not in softmaxScoresPerHeader:
                    softmaxScoresPerHeader[header] = getSoftmax(model,tokenizer,tokens=tokenChunk,n=topScoresToPrint, printTopN=printTopN, printRawScores=printRawScores)
                else:
                    softmaxScoresPerHeader[header] += getSoftmax(model,tokenizer,tokens=tokenChunk,n=topScoresToPrint, printTopN=printTopN, printRawScores=printRawScores)
            # ^^ I could encode the tokens in batches, but there aren't many to encode so it shouldn't be a problem
            
    #Part 3 determine what to do with the neutral labels

    moodLabel = convertScoresToLabels(softmaxScoresPerHeader,freqs, emotionsAsValenceArousal,emotion_dict,disregardNeutral=disregardNeutral)
        #^^ rewrite function to account for freqs of header and disregard softmaxes with neutral labels as top score!


    #part 4 - return the most common label
    return moodLabel

## Testing Function

In [486]:
getMoodLabelFromLyrics(fullLyricsLilTecca,model, tokenizer, emotion_dict, emotionsAsValenceArousal,disregardNeutral=True, printRawScores=False, printTopN=False,topScoresToPrint=3,max_length=512, device="cuda",  returnSongSegments=False)

'anxious'